<a href="https://colab.research.google.com/github/FowadIjaz/Portfolio/blob/main/Geodataframe_Heatmaps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Overview**

*This heatmap was completed by Fowad Ijaz for M733: Marketing Analytics taught by K. Deal at the DeGroote School of Business in Fall 2020.*

The purpose of this project was to create a heatmap of Ontario using postal codes. Using [this code](https://stackoverflow.com/questions/58043978/display-data-on-real-map-based-on-postal-code) as a starting point, I created a heatmap of Ontario that could be broken into segments based on Canadian Census data. The data was then mapped onto boundary files provided by the Government of Canada on the [StatsCan](https://www12.statcan.gc.ca/census-recensement/2011/geo/bound-limit/bound-limit-2011-eng.cfm) website. 

The main challenge was coercing the data into a geodataframe such that the geometry of the shapefile could be read in by the graphing packages. After the mapping was done, various heatmaps were generated based on postal code-level responses to questions like "how high is your confidence in big businesses?" and "how enthusiastic are you about new technologies?" 

While the responses were initially provided for 6 digit postal codes (FSALDU), they had to be generalized to the 3 digit postal codes (FSA/CFSAUID). This was done because the Canada post doesn't currently make FSALDU boundaries available without a license. Despite this, the resulting plots were illustrative of general trends in sentiments based on geographic location. 


In [1]:
pip install pandas_bokeh

In [2]:
pip install geopandas

     |████████████████████████████████| 972kB 8.3MB/s 
     |████████████████████████████████| 6.5MB 13.3MB/s 
     |████████████████████████████████| 14.8MB 314kB/s 


In [9]:
#importing necessary packages
import geopandas
import pandas as pd
import pandas_bokeh
import matplotlib.pyplot as plt
pandas_bokeh.output_notebook()

canada = geopandas.read_file("/content/gfsa000b11a_e.shp")
ontario = canada[canada['PRUID'] == '35']

df= pd.read_csv('/content/FS_4.csv')
df['FSA'] = df['CODE'].str[:3]

In [10]:
# displaying the dataframe to ensure it was read in correctly
df

,Confidence,predict,Tech.Enthu.,Ann_Prem_Hm_Insu,Mob_Mark_cons,Mob_Inf_sec_cons,X..Smart_purch,X.Smt_Therm,X.Smt_Hm_Secu,X.Smt_Hm_Ass,Smt_Hm_Dev,X.Smat_Therm.Pln,X.Smt_Hm_Sec.Pln,X.Smt_Hm_Asst.Pln,X.Smt_Hm_Dev.Pln,Fin_App_Trst,CODE,Pop_Age,M_Pop_Age,F_Pop_Age,X._Pvt_HHld,X._Fam_Hhld,X._NF_Hhld,Avg_Chld_PrCen_Fm_Hhld,Houses,Apt,Condos,Hhld.Income,High.School,Clg_CEGEP_Non_Uni_Dip,Uni_Dip_Blw_Bach,Empl,UnEmp,Tot_Mig,FSA
0,0.009427,Seg2,0.009740,0.004404,0.005647,0.007529,0.007153,0.009788,0.004518,0.012047,0.007153,0.004518,0.007906,0.002635,0.004141,0.018824,K7L5R2,41.8,42.1,15.0,6.0,2,1,3.0,0.002353,0.002353,0.002353,127.502612,0.007059,0.009412,0.014118,0.011765,0.004706,0.023529,K7L
1,0.025949,Seg2,0.014751,0.005866,0.012673,0.021386,0.014257,0.022178,0.007129,0.025347,0.011881,0.011089,0.012673,0.005545,0.011089,0.045149,K9V6H4,32.9,32.3,37.5,11.0,1,0,3.0,0.009901,0.000000,0.000000,905.853366,0.039604,0.019802,0.000000,0.049505,0.000000,0.009901,K9V
2,0.025238,Seg2,0.029874,0.020745,0.015556,0.025556,0.018889,0.028889,0.012222,0.032222,0.018889,0.011111,0.018889,0.011111,0.008889,0.056667,N2L2G4,84.3,83.6,85.0,7.0,1,0,2.0,0.027778,0.000000,0.000000,2407.275000,0.000000,0.000000,0.083333,0.055556,0.027778,0.027778,N2L
3,0.031883,Seg2,0.024818,0.018099,0.011765,0.027059,0.021176,0.028235,0.009412,0.035294,0.020000,0.016471,0.014118,0.015294,0.012941,0.065882,N2L2J2,85.0,86.1,83.5,5.0,1,0,1.0,0.029412,0.000000,0.000000,1528.112941,0.029412,0.029412,0.029412,0.088235,0.029412,0.029412,N2L
4,0.030434,Seg2,0.028480,0.009048,0.018765,0.027654,0.024691,0.023704,0.011852,0.026667,0.018765,0.017778,0.021728,0.012840,0.015802,0.058272,M6B2P9,83.8,76.9,86.0,14.0,1,0,3.0,0.012346,0.000000,0.000000,1764.748642,0.049383,0.012346,0.012346,0.086420,0.000000,0.074074,M6B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252721,0.161750,Seg1,0.268275,0.084187,0.210000,0.420000,0.220000,0.130000,0.070000,0.180000,0.130000,0.120000,0.080000,0.130000,0.080000,0.550000,K9J3E9,32.8,37.2,22.5,3.5,1,1,2.0,0.142857,0.142857,0.000000,5714.285714,0.428571,0.285714,0.000000,0.571429,0.000000,0.000000,K9J
252722,0.255816,Seg3,0.309055,0.055744,0.270000,0.420000,0.380000,0.220000,0.110000,0.320000,0.200000,0.120000,0.180000,0.110000,0.160000,0.630000,M5V0R3,30.6,30.8,30.0,9.0,0,1,0.0,0.000000,0.111111,0.111111,50474.781110,0.111111,0.111111,0.777778,0.888889,0.111111,0.222222,M5V
252723,0.161750,Seg1,0.268275,0.163696,0.230000,0.420000,0.290000,0.110000,0.060000,0.170000,0.150000,0.060000,0.060000,0.090000,0.080000,0.600000,N3T3V6,37.2,25.0,47.5,1.8,3,2,0.7,0.333333,0.222222,0.000000,1717.035556,0.444444,0.222222,0.111111,0.444444,0.000000,0.222222,N3T
252724,0.255816,Seg3,0.309055,0.079215,0.280000,0.420000,0.430000,0.200000,0.080000,0.340000,0.250000,0.170000,0.200000,0.120000,0.110000,0.610000,M5A1M1,32.4,35.0,28.8,6.3,1,2,0.0,0.105263,0.052632,0.052632,7565.789474,0.157895,0.157895,0.578947,0.894737,0.052632,0.210526,M5A


In [11]:
# displaying the ontario geometry data to ensure it was read in correctly
ontario

,CFSAUID,PRUID,PRNAME,geometry
1093,K0A,35,Ontario,"MULTIPOLYGON (((-76.29379 45.12423, -76.29155 ..."
1094,K0B,35,Ontario,"POLYGON ((-74.62553 45.63457, -74.62996 45.625..."
1095,K0C,35,Ontario,"MULTIPOLYGON (((-75.18802 44.88576, -75.18851 ..."
1096,K0E,35,Ontario,"MULTIPOLYGON (((-76.17936 44.29474, -76.17917 ..."
1097,K0G,35,Ontario,"MULTIPOLYGON (((-76.05719 44.83265, -76.05783 ..."
...,...,...,...,...
1604,P7L,35,Ontario,"MULTIPOLYGON (((-89.49600 48.02922, -89.49605 ..."
1605,P8N,35,Ontario,"MULTIPOLYGON (((-92.34525 49.58323, -92.34441 ..."
1606,P8T,35,Ontario,"POLYGON ((-91.57620 50.57053, -91.58012 50.571..."
1607,P9A,35,Ontario,"MULTIPOLYGON (((-91.95512 48.40489, -91.95511 ..."


In [12]:
# aggregating the dataframe by the 3 digit postal codes
df1 = df.groupby(['FSA']).agg(['mean'])
df1.columns = df1.columns.droplevel(1)
df1


,Confidence,Tech.Enthu.,Ann_Prem_Hm_Insu,Mob_Mark_cons,Mob_Inf_sec_cons,X..Smart_purch,X.Smt_Therm,X.Smt_Hm_Secu,X.Smt_Hm_Ass,Smt_Hm_Dev,X.Smat_Therm.Pln,X.Smt_Hm_Sec.Pln,X.Smt_Hm_Asst.Pln,X.Smt_Hm_Dev.Pln,Fin_App_Trst,Pop_Age,M_Pop_Age,F_Pop_Age,X._Pvt_HHld,X._Fam_Hhld,X._NF_Hhld,Avg_Chld_PrCen_Fm_Hhld,Houses,Apt,Condos,Hhld.Income,High.School,Clg_CEGEP_Non_Uni_Dip,Uni_Dip_Blw_Bach,Empl,UnEmp,Tot_Mig
FSA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
K0A,0.194100,0.181877,0.267542,0.123170,0.146123,0.101577,0.175693,0.062779,0.178321,0.103460,0.089208,0.104465,0.061874,0.073228,0.391425,42.447222,42.313889,42.830556,2.661111,946.583333,256.055556,1.025000,0.336464,0.037897,0.011838,84.341674,0.234643,0.220292,0.176579,0.555569,0.027013,0.081208
K0B,0.223986,0.243886,0.305832,0.123400,0.138210,0.092094,0.125235,0.036067,0.158949,0.070453,0.079820,0.095384,0.043547,0.064107,0.388569,49.369231,49.115385,49.592308,2.300000,536.538462,202.692308,0.861538,0.397090,0.031158,0.010003,78.974515,0.258007,0.194857,0.116304,0.512899,0.023876,0.051056
K0C,0.181314,0.136814,0.269570,0.134266,0.132031,0.087865,0.142359,0.046564,0.172251,0.084453,0.072086,0.084425,0.052106,0.058468,0.368899,47.150000,46.542857,47.803571,2.564286,571.607143,205.500000,0.967857,0.367493,0.028169,0.006691,72.451199,0.249898,0.222232,0.113130,0.494427,0.028419,0.057387
K0E,0.189414,0.144480,0.287732,0.120085,0.126200,0.085871,0.152505,0.055350,0.162904,0.084477,0.062383,0.078416,0.047887,0.067378,0.398885,49.780000,49.345000,50.190000,2.335000,619.450000,213.150000,0.850000,0.401049,0.022828,0.003053,146.720447,0.262198,0.228530,0.118137,0.461231,0.031899,0.057499
K0G,0.181943,0.139536,0.278134,0.127117,0.132117,0.086241,0.150936,0.046518,0.153881,0.097499,0.076045,0.089080,0.054696,0.072891,0.414314,50.543750,50.293750,50.787500,2.431250,779.437500,252.375000,0.837500,0.390774,0.026374,0.005301,91.217291,0.247289,0.244795,0.139901,0.451323,0.027984,0.066393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P7L,0.192317,0.131406,0.254652,0.117310,0.159553,0.092008,0.178852,0.073483,0.191222,0.098343,0.086622,0.084125,0.052683,0.081577,0.438275,48.925000,50.529545,44.406818,2.536364,15.386364,4.272727,0.881818,0.403163,0.001487,0.000000,3028.917224,0.239337,0.244499,0.141294,0.533431,0.024480,0.042095
P8N,0.223876,0.196872,0.254172,0.129753,0.172549,0.107517,0.156861,0.056242,0.175737,0.091717,0.081888,0.099708,0.068584,0.088589,0.423830,44.910959,43.430411,37.337534,2.484384,8.783562,4.216438,1.069041,0.352810,0.083843,0.000636,10939.486578,0.270507,0.235406,0.115432,0.577711,0.032018,0.062230
P8T,0.251216,0.276823,0.195214,0.125944,0.164836,0.118495,0.142879,0.055328,0.157811,0.081625,0.075780,0.086334,0.066693,0.089789,0.390863,35.884483,34.631034,36.136207,2.739655,26.379310,10.620690,1.286207,0.250588,0.097571,0.005322,2083.796412,0.207412,0.201190,0.131504,0.536471,0.032337,0.036131


In [13]:
# creating a dataframe for segment analysis 

df1a = df.groupby(['FSA']).agg(pd.Series.mode)
df1b = df1a['predict']

df1b


FSA
K0A    Seg2
K0B    Seg2
K0C    Seg2
K0E    Seg2
K0G    Seg2
       ... 
P7L    Seg2
P8N    Seg2
P8T    Seg2
P9A    Seg2
P9N    Seg2
Name: predict, Length: 518, dtype: object

In [14]:
# merging the shapefile containing Ontario's geometry data with the dataframe containing the other response variables. 

df2 = pd.merge(ontario, df1, left_on='CFSAUID', right_index=True)
df2

,CFSAUID,PRUID,PRNAME,geometry,Confidence,Tech.Enthu.,Ann_Prem_Hm_Insu,Mob_Mark_cons,Mob_Inf_sec_cons,X..Smart_purch,X.Smt_Therm,X.Smt_Hm_Secu,X.Smt_Hm_Ass,Smt_Hm_Dev,X.Smat_Therm.Pln,X.Smt_Hm_Sec.Pln,X.Smt_Hm_Asst.Pln,X.Smt_Hm_Dev.Pln,Fin_App_Trst,Pop_Age,M_Pop_Age,F_Pop_Age,X._Pvt_HHld,X._Fam_Hhld,X._NF_Hhld,Avg_Chld_PrCen_Fm_Hhld,Houses,Apt,Condos,Hhld.Income,High.School,Clg_CEGEP_Non_Uni_Dip,Uni_Dip_Blw_Bach,Empl,UnEmp,Tot_Mig
1093,K0A,35,Ontario,"MULTIPOLYGON (((-76.29379 45.12423, -76.29155 ...",0.194100,0.181877,0.267542,0.123170,0.146123,0.101577,0.175693,0.062779,0.178321,0.103460,0.089208,0.104465,0.061874,0.073228,0.391425,42.447222,42.313889,42.830556,2.661111,946.583333,256.055556,1.025000,0.336464,0.037897,0.011838,84.341674,0.234643,0.220292,0.176579,0.555569,0.027013,0.081208
1094,K0B,35,Ontario,"POLYGON ((-74.62553 45.63457, -74.62996 45.625...",0.223986,0.243886,0.305832,0.123400,0.138210,0.092094,0.125235,0.036067,0.158949,0.070453,0.079820,0.095384,0.043547,0.064107,0.388569,49.369231,49.115385,49.592308,2.300000,536.538462,202.692308,0.861538,0.397090,0.031158,0.010003,78.974515,0.258007,0.194857,0.116304,0.512899,0.023876,0.051056
1095,K0C,35,Ontario,"MULTIPOLYGON (((-75.18802 44.88576, -75.18851 ...",0.181314,0.136814,0.269570,0.134266,0.132031,0.087865,0.142359,0.046564,0.172251,0.084453,0.072086,0.084425,0.052106,0.058468,0.368899,47.150000,46.542857,47.803571,2.564286,571.607143,205.500000,0.967857,0.367493,0.028169,0.006691,72.451199,0.249898,0.222232,0.113130,0.494427,0.028419,0.057387
1096,K0E,35,Ontario,"MULTIPOLYGON (((-76.17936 44.29474, -76.17917 ...",0.189414,0.144480,0.287732,0.120085,0.126200,0.085871,0.152505,0.055350,0.162904,0.084477,0.062383,0.078416,0.047887,0.067378,0.398885,49.780000,49.345000,50.190000,2.335000,619.450000,213.150000,0.850000,0.401049,0.022828,0.003053,146.720447,0.262198,0.228530,0.118137,0.461231,0.031899,0.057499
1097,K0G,35,Ontario,"MULTIPOLYGON (((-76.05719 44.83265, -76.05783 ...",0.181943,0.139536,0.278134,0.127117,0.132117,0.086241,0.150936,0.046518,0.153881,0.097499,0.076045,0.089080,0.054696,0.072891,0.414314,50.543750,50.293750,50.787500,2.431250,779.437500,252.375000,0.837500,0.390774,0.026374,0.005301,91.217291,0.247289,0.244795,0.139901,0.451323,0.027984,0.066393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1604,P7L,35,Ontario,"MULTIPOLYGON (((-89.49600 48.02922, -89.49605 ...",0.192317,0.131406,0.254652,0.117310,0.159553,0.092008,0.178852,0.073483,0.191222,0.098343,0.086622,0.084125,0.052683,0.081577,0.438275,48.925000,50.529545,44.406818,2.536364,15.386364,4.272727,0.881818,0.403163,0.001487,0.000000,3028.917224,0.239337,0.244499,0.141294,0.533431,0.024480,0.042095
1605,P8N,35,Ontario,"MULTIPOLYGON (((-92.34525 49.58323, -92.34441 ...",0.223876,0.196872,0.254172,0.129753,0.172549,0.107517,0.156861,0.056242,0.175737,0.091717,0.081888,0.099708,0.068584,0.088589,0.423830,44.910959,43.430411,37.337534,2.484384,8.783562,4.216438,1.069041,0.352810,0.083843,0.000636,10939.486578,0.270507,0.235406,0.115432,0.577711,0.032018,0.062230
1606,P8T,35,Ontario,"POLYGON ((-91.57620 50.57053, -91.58012 50.571...",0.251216,0.276823,0.195214,0.125944,0.164836,0.118495,0.142879,0.055328,0.157811,0.081625,0.075780,0.086334,0.066693,0.089789,0.390863,35.884483,34.631034,36.136207,2.739655,26.379310,10.620690,1.286207,0.250588,0.097571,0.005322,2083.796412,0.207412,0.201190,0.131504,0.536471,0.032337,0.036131
1607,P9A,35,Ontario,"MULTIPOLYGON (((-91.95512 48.40489, -91.95511 ...",0.212026,0.175243,0.253891,0.127051,0.168856,0.103946,0.158355,0.056132,0.170560,0.088507,0.076080,0.095871,0.064794,0.085490,0.416118,43.991106,41.717308,41.481731,2.527644,6.240385,3.478365,1.154567,0.365585,0.066904,0.004079,8753.657734,0.265601,0.230194,0.114242,0.551463,0.019608,0.031531


In [15]:
# Merging the predict variable back into the dataframe 
df3 = pd.merge(df2, df1b, left_on='CFSAUID', right_index=True)
df3

,CFSAUID,PRUID,PRNAME,geometry,Confidence,Tech.Enthu.,Ann_Prem_Hm_Insu,Mob_Mark_cons,Mob_Inf_sec_cons,X..Smart_purch,X.Smt_Therm,X.Smt_Hm_Secu,X.Smt_Hm_Ass,Smt_Hm_Dev,X.Smat_Therm.Pln,X.Smt_Hm_Sec.Pln,X.Smt_Hm_Asst.Pln,X.Smt_Hm_Dev.Pln,Fin_App_Trst,Pop_Age,M_Pop_Age,F_Pop_Age,X._Pvt_HHld,X._Fam_Hhld,X._NF_Hhld,Avg_Chld_PrCen_Fm_Hhld,Houses,Apt,Condos,Hhld.Income,High.School,Clg_CEGEP_Non_Uni_Dip,Uni_Dip_Blw_Bach,Empl,UnEmp,Tot_Mig,predict
1093,K0A,35,Ontario,"MULTIPOLYGON (((-76.29379 45.12423, -76.29155 ...",0.194100,0.181877,0.267542,0.123170,0.146123,0.101577,0.175693,0.062779,0.178321,0.103460,0.089208,0.104465,0.061874,0.073228,0.391425,42.447222,42.313889,42.830556,2.661111,946.583333,256.055556,1.025000,0.336464,0.037897,0.011838,84.341674,0.234643,0.220292,0.176579,0.555569,0.027013,0.081208,Seg2
1094,K0B,35,Ontario,"POLYGON ((-74.62553 45.63457, -74.62996 45.625...",0.223986,0.243886,0.305832,0.123400,0.138210,0.092094,0.125235,0.036067,0.158949,0.070453,0.079820,0.095384,0.043547,0.064107,0.388569,49.369231,49.115385,49.592308,2.300000,536.538462,202.692308,0.861538,0.397090,0.031158,0.010003,78.974515,0.258007,0.194857,0.116304,0.512899,0.023876,0.051056,Seg2
1095,K0C,35,Ontario,"MULTIPOLYGON (((-75.18802 44.88576, -75.18851 ...",0.181314,0.136814,0.269570,0.134266,0.132031,0.087865,0.142359,0.046564,0.172251,0.084453,0.072086,0.084425,0.052106,0.058468,0.368899,47.150000,46.542857,47.803571,2.564286,571.607143,205.500000,0.967857,0.367493,0.028169,0.006691,72.451199,0.249898,0.222232,0.113130,0.494427,0.028419,0.057387,Seg2
1096,K0E,35,Ontario,"MULTIPOLYGON (((-76.17936 44.29474, -76.17917 ...",0.189414,0.144480,0.287732,0.120085,0.126200,0.085871,0.152505,0.055350,0.162904,0.084477,0.062383,0.078416,0.047887,0.067378,0.398885,49.780000,49.345000,50.190000,2.335000,619.450000,213.150000,0.850000,0.401049,0.022828,0.003053,146.720447,0.262198,0.228530,0.118137,0.461231,0.031899,0.057499,Seg2
1097,K0G,35,Ontario,"MULTIPOLYGON (((-76.05719 44.83265, -76.05783 ...",0.181943,0.139536,0.278134,0.127117,0.132117,0.086241,0.150936,0.046518,0.153881,0.097499,0.076045,0.089080,0.054696,0.072891,0.414314,50.543750,50.293750,50.787500,2.431250,779.437500,252.375000,0.837500,0.390774,0.026374,0.005301,91.217291,0.247289,0.244795,0.139901,0.451323,0.027984,0.066393,Seg2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1604,P7L,35,Ontario,"MULTIPOLYGON (((-89.49600 48.02922, -89.49605 ...",0.192317,0.131406,0.254652,0.117310,0.159553,0.092008,0.178852,0.073483,0.191222,0.098343,0.086622,0.084125,0.052683,0.081577,0.438275,48.925000,50.529545,44.406818,2.536364,15.386364,4.272727,0.881818,0.403163,0.001487,0.000000,3028.917224,0.239337,0.244499,0.141294,0.533431,0.024480,0.042095,Seg2
1605,P8N,35,Ontario,"MULTIPOLYGON (((-92.34525 49.58323, -92.34441 ...",0.223876,0.196872,0.254172,0.129753,0.172549,0.107517,0.156861,0.056242,0.175737,0.091717,0.081888,0.099708,0.068584,0.088589,0.423830,44.910959,43.430411,37.337534,2.484384,8.783562,4.216438,1.069041,0.352810,0.083843,0.000636,10939.486578,0.270507,0.235406,0.115432,0.577711,0.032018,0.062230,Seg2
1606,P8T,35,Ontario,"POLYGON ((-91.57620 50.57053, -91.58012 50.571...",0.251216,0.276823,0.195214,0.125944,0.164836,0.118495,0.142879,0.055328,0.157811,0.081625,0.075780,0.086334,0.066693,0.089789,0.390863,35.884483,34.631034,36.136207,2.739655,26.379310,10.620690,1.286207,0.250588,0.097571,0.005322,2083.796412,0.207412,0.201190,0.131504,0.536471,0.032337,0.036131,Seg2
1607,P9A,35,Ontario,"MULTIPOLYGON (((-91.95512 48.40489, -91.95511 ...",0.212026,0.175243,0.253891,0.127051,0.168856,0.103946,0.158355,0.056132,0.170560,0.088507,0.076080,0.095871,0.064794,0.085490,0.416118,43.991106,41.717308,41.481731,2.527644,6.240385,3.478365,1.154567,0.365585,0.066904,0.004079,8753.657734,0.265601,0.230194,0.114242,0.551463,0.019608,0.031531,Seg2


In [16]:
# switching the Predict variable type to numeric so that it can be plotted
df3['predict'] = df3['predict'].str[3:]
df3['predict'] = pd.to_numeric(df3['predict'])

df3

,CFSAUID,PRUID,PRNAME,geometry,Confidence,Tech.Enthu.,Ann_Prem_Hm_Insu,Mob_Mark_cons,Mob_Inf_sec_cons,X..Smart_purch,X.Smt_Therm,X.Smt_Hm_Secu,X.Smt_Hm_Ass,Smt_Hm_Dev,X.Smat_Therm.Pln,X.Smt_Hm_Sec.Pln,X.Smt_Hm_Asst.Pln,X.Smt_Hm_Dev.Pln,Fin_App_Trst,Pop_Age,M_Pop_Age,F_Pop_Age,X._Pvt_HHld,X._Fam_Hhld,X._NF_Hhld,Avg_Chld_PrCen_Fm_Hhld,Houses,Apt,Condos,Hhld.Income,High.School,Clg_CEGEP_Non_Uni_Dip,Uni_Dip_Blw_Bach,Empl,UnEmp,Tot_Mig,predict
1093,K0A,35,Ontario,"MULTIPOLYGON (((-76.29379 45.12423, -76.29155 ...",0.194100,0.181877,0.267542,0.123170,0.146123,0.101577,0.175693,0.062779,0.178321,0.103460,0.089208,0.104465,0.061874,0.073228,0.391425,42.447222,42.313889,42.830556,2.661111,946.583333,256.055556,1.025000,0.336464,0.037897,0.011838,84.341674,0.234643,0.220292,0.176579,0.555569,0.027013,0.081208,2.0
1094,K0B,35,Ontario,"POLYGON ((-74.62553 45.63457, -74.62996 45.625...",0.223986,0.243886,0.305832,0.123400,0.138210,0.092094,0.125235,0.036067,0.158949,0.070453,0.079820,0.095384,0.043547,0.064107,0.388569,49.369231,49.115385,49.592308,2.300000,536.538462,202.692308,0.861538,0.397090,0.031158,0.010003,78.974515,0.258007,0.194857,0.116304,0.512899,0.023876,0.051056,2.0
1095,K0C,35,Ontario,"MULTIPOLYGON (((-75.18802 44.88576, -75.18851 ...",0.181314,0.136814,0.269570,0.134266,0.132031,0.087865,0.142359,0.046564,0.172251,0.084453,0.072086,0.084425,0.052106,0.058468,0.368899,47.150000,46.542857,47.803571,2.564286,571.607143,205.500000,0.967857,0.367493,0.028169,0.006691,72.451199,0.249898,0.222232,0.113130,0.494427,0.028419,0.057387,2.0
1096,K0E,35,Ontario,"MULTIPOLYGON (((-76.17936 44.29474, -76.17917 ...",0.189414,0.144480,0.287732,0.120085,0.126200,0.085871,0.152505,0.055350,0.162904,0.084477,0.062383,0.078416,0.047887,0.067378,0.398885,49.780000,49.345000,50.190000,2.335000,619.450000,213.150000,0.850000,0.401049,0.022828,0.003053,146.720447,0.262198,0.228530,0.118137,0.461231,0.031899,0.057499,2.0
1097,K0G,35,Ontario,"MULTIPOLYGON (((-76.05719 44.83265, -76.05783 ...",0.181943,0.139536,0.278134,0.127117,0.132117,0.086241,0.150936,0.046518,0.153881,0.097499,0.076045,0.089080,0.054696,0.072891,0.414314,50.543750,50.293750,50.787500,2.431250,779.437500,252.375000,0.837500,0.390774,0.026374,0.005301,91.217291,0.247289,0.244795,0.139901,0.451323,0.027984,0.066393,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1604,P7L,35,Ontario,"MULTIPOLYGON (((-89.49600 48.02922, -89.49605 ...",0.192317,0.131406,0.254652,0.117310,0.159553,0.092008,0.178852,0.073483,0.191222,0.098343,0.086622,0.084125,0.052683,0.081577,0.438275,48.925000,50.529545,44.406818,2.536364,15.386364,4.272727,0.881818,0.403163,0.001487,0.000000,3028.917224,0.239337,0.244499,0.141294,0.533431,0.024480,0.042095,2.0
1605,P8N,35,Ontario,"MULTIPOLYGON (((-92.34525 49.58323, -92.34441 ...",0.223876,0.196872,0.254172,0.129753,0.172549,0.107517,0.156861,0.056242,0.175737,0.091717,0.081888,0.099708,0.068584,0.088589,0.423830,44.910959,43.430411,37.337534,2.484384,8.783562,4.216438,1.069041,0.352810,0.083843,0.000636,10939.486578,0.270507,0.235406,0.115432,0.577711,0.032018,0.062230,2.0
1606,P8T,35,Ontario,"POLYGON ((-91.57620 50.57053, -91.58012 50.571...",0.251216,0.276823,0.195214,0.125944,0.164836,0.118495,0.142879,0.055328,0.157811,0.081625,0.075780,0.086334,0.066693,0.089789,0.390863,35.884483,34.631034,36.136207,2.739655,26.379310,10.620690,1.286207,0.250588,0.097571,0.005322,2083.796412,0.207412,0.201190,0.131504,0.536471,0.032337,0.036131,2.0
1607,P9A,35,Ontario,"MULTIPOLYGON (((-91.95512 48.40489, -91.95511 ...",0.212026,0.175243,0.253891,0.127051,0.168856,0.103946,0.158355,0.056132,0.170560,0.088507,0.076080,0.095871,0.064794,0.085490,0.416118,43.991106,41.717308,41.481731,2.527644,6.240385,3.478365,1.154567,0.365585,0.066904,0.004079,8753.657734,0.265601,0.230194,0.114242,0.551463,0.019608,0.031531,2.0


In [17]:
df3.info()

# df2.geometry.value_counts

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 513 entries, 1093 to 1608
Data columns (total 37 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   CFSAUID                 513 non-null    object  
 1   PRUID                   513 non-null    object  
 2   PRNAME                  513 non-null    object  
 3   geometry                513 non-null    geometry
 4   Confidence              513 non-null    float64 
 5   Tech.Enthu.             513 non-null    float64 
 6   Ann_Prem_Hm_Insu        513 non-null    float64 
 7   Mob_Mark_cons           513 non-null    float64 
 8   Mob_Inf_sec_cons        513 non-null    float64 
 9   X..Smart_purch          513 non-null    float64 
 10  X.Smt_Therm             513 non-null    float64 
 11  X.Smt_Hm_Secu           513 non-null    float64 
 12  X.Smt_Hm_Ass            513 non-null    float64 
 13  Smt_Hm_Dev              513 non-null    float64 
 14  X.Smat_Therm.P

In [18]:
# # Join ontario dataset with sample data
# new_df=ontario.join(df.set_index('FSA'), on='CFSAUID')

# new_df
# df2.plot()

# new_df2 = new_df.groupby(['CFSAUID']).agg(['count'])
# new_df2

# new_df2 = new_df.groupby(['CFSAUID']).agg(pd.Series.mode)
# new_df2

# new_df2 = new_df.groupby(['geometry']).agg(['count','max'])
# new_df2

### **Results**

*Confidence in Big Business*

The plot below shows that confidence in big business is not higher than 40% anywhere, but is consistently lower in Southern Ontario.

In [19]:
df2.plot_bokeh(simplify_shapes=1000,
                  category="Confidence", 
                  colormap="Spectral", 
                  hovertool_columns=["CFSAUID","Confidence"])

Figure(id='1801', ...)

*Tech Enthusiasm*

Surprisingly, tech enthusiasm appears to be highest in Northern Ontario. This could be due to a number of factors including small sample size, or possibly because a remote population is able to understand the role that technology is able to play in enhancing people's quality of life. 

Another major hotspot with high tech enthusiasm is, of course, Toronto. When zoomed in, we can see that it is consistently higher than its surrounding regions. 

In [20]:

df2.plot_bokeh(simplify_shapes=1000,
                  category="Tech.Enthu.", 
                  colormap="Spectral", 
                  hovertool_columns=["CFSAUID","Tech.Enthu."])


Figure(id='1941', ...)

In [8]:
# Setting simplify_shapes to a lower amount increases the visual fidelity at the cost of processing speed
# Showing a sample heatmap with a simplify_shapes value of 500, meaning it is accurate to 500 metres

df=pd.DataFrame({'PCODE': ['P0V','P0L','P0T','P0Y', 'P0G', 'P2N'], 'A':[6,3,5,2,2,4] })

# Join ontario dataset with sample data
new_df=ontario.join(df.set_index('PCODE'), on='CFSAUID')


new_df.plot_bokeh(simplify_shapes=500,
                  category="A", 
                  colormap="Spectral", 
                  hovertool_columns=["CFSAUID","A"])

Figure(id='1621', ...)